In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests as req
import json
from urllib.parse import quote

In [29]:
transactions=pd.read_csv('input/transactions.csv',sep='|')
display(transactions.head(20))
items=pd.read_csv('input/items_int_copy_copy.csv',sep='|')
display(items.head(20))
transactions.columns = transactions.columns.str.replace(' ','_')
items.columns = items.columns.str.replace(' ','_')

,sessionID,itemID,click,basket,order
0,0,21310,1,0,0
1,1,73018,1,0,0
2,2,19194,1,0,0
3,3,40250,1,0,0
4,4,46107,1,0,0
5,5,34217,1,0,0
6,6,31436,1,0,0
7,7,14576,1,1,0
8,7,17731,2,1,0
9,8,58723,1,0,0


,itemID,title,author,publisher,main topic,subtopics
0,71208,Jack and the Beanstalk,Robin Twiddy,FUSION PR,YFJ,[]
1,37181,When I'm Too Excited: English Edition,Inhabit Education Books,INHABIT EDUCATION BOOKS INC,YXE,[YF]
2,36246,When I'm Sad: English Edition,Inhabit Education Books,INHABIT EDUCATION BOOKS INC,YXE,[YF]
3,27706,The Wheels on the Bus: A Moving Animation,Cider Mill Press,APPLESAUCE PR,YF,[]
4,21726,Devoratoarea de sentimente,Alina Moldovanu,Lulu.com,FM,[]
5,34275,Ollie's Backpack,Riya Aarini,Riya Aarini,YFB,[YXE]
6,77135,The Diamond in the Window,Jane Langton,PURPLE HOUSE PR,YFC,"[YFH,YFN]"
7,49606,Butch T. Cougar: Mascot or Superhero?,Caryn Lawton,WASHINGTON STATE UNIV PR,YFH,"[YFP,YFQ]"
8,32692,The Records of Erevalon,K. A. BUTLER,Austin Macauley,YFH,[]
9,63393,Crônicas de Damocles: Gusmão,Vinicius Watzl,LIGHTNING SOURCE INC,FL,[]


In [3]:
#Splitting the subtopics
items['subtopics']=items['subtopics'].str[1:-1]
#l1=[str(x) for x in range(31)]
l2=["subtopic"+str(x) for x in range(31)]
#rename_dict={l1[x]:l2[x] for x in range(31)}
items[l2]=items.subtopics.str.split(expand=True,pat=",")
#items2.rename(columns=rename_dict,inplace=True)

In [4]:
#Just a brief numerical summary of the transactions data
print(transactions.describe())
print("*************")
print(transactions.shape)
print("*************")
print(transactions.isna().sum())

           sessionID         itemID          click         basket  \
count  365143.000000  365143.000000  365143.000000  365143.000000   
mean   139586.939175   40051.292307       1.233180       0.141202   
std     80795.207871   22493.347334       1.069996       1.107574   
min         0.000000       1.000000       0.000000       0.000000   
25%     69459.500000   20713.000000       1.000000       0.000000   
50%    139608.000000   40692.000000       1.000000       0.000000   
75%    209750.500000   58916.000000       1.000000       0.000000   
max    279354.000000   79066.000000     118.000000     293.000000   

               order  
count  365143.000000  
mean        0.048403  
std         0.268717  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max        28.000000  
*************
(365143, 5)
*************
sessionID    0
itemID       0
click        0
basket       0
order        0
dtype: int64


In [11]:
#Just a brief summary of items data
print(items.describe())
print("*************")
print(items.shape)
print("*************")
items=items.replace(r'^\s*$', np.nan, regex=True)
print((items.isna().sum()/78030)*100)

             itemID
count  15531.000000
mean   39428.412208
std    22858.122544
min        3.000000
25%    19607.500000
50%    39270.000000
75%    59005.500000
max    79048.000000
*************
(15531, 6)
*************
itemID        0.000000
title         0.000000
author        0.535691
publisher     0.010252
main_topic    0.010252
subtopics     0.001282
dtype: float64


In [6]:
#Checking for qualifiers
total_non_na=len(items)-items.subtopics.isna().sum()
print("Total Non-NA sub topics " + str(total_non_na))
for i in range(1,7):
    formatted_float = "{:.2f}".format(len(items[items["subtopics"].str.contains(str(i),na=False)])/total_non_na*100)
    print("For qualifier "+str(i) +" - "+ formatted_float+"%")
    


Total Non-NA sub topics 41125
For qualifier 1 - 14.61%
For qualifier 2 - 2.04%
For qualifier 3 - 3.95%
For qualifier 4 - 4.32%
For qualifier 5 - 35.42%
For qualifier 6 - 0.67%


In [7]:
total_non_na=len(items)-items.main_topic.isna().sum()
for i in range(1,7):
    formatted_float = "{:.2f}".format(len(items[items["main_topic"].str.contains(str(i),na=False) | items["subtopics"].str.contains(str(i),na=False)])/total_non_na*100)
    print("For qualifier "+str(i) +" - "+ formatted_float+"%")

For qualifier 1 - 8.73%
For qualifier 2 - 1.90%
For qualifier 3 - 2.39%
For qualifier 4 - 2.34%
For qualifier 5 - 18.76%
For qualifier 6 - 0.36%


In [8]:
total_non_na=len(items)-items.main_topic.isna().sum()
print("Total Non-NA main topics " + str(total_non_na))
for i in range(1,7):
    formatted_float = "{:.2f}".format(len(items[items["main_topic"].str.contains(str(i),na=False)])/total_non_na*100)
    print("For qualifier "+str(i) +" - "+ formatted_float+"%")

Total Non-NA main topics 77772
For qualifier 1 - 1.26%
For qualifier 2 - 1.33%
For qualifier 3 - 0.33%
For qualifier 4 - 0.15%
For qualifier 5 - 0.05%
For qualifier 6 - 0.02%


In [9]:
#Further diving into the language qualifiers for sub topics
language_categories=["A","B","C","F","G","H","J","P","S","Z"]
total_non_na=len(items)-items.subtopics.isna().sum()
print("Total Non-NA sub topics " + str(total_non_na))
for i in range(len(language_categories)):
    formatted_float = "{:.2f}".format(len(items[items["subtopics"].str.contains("2"+language_categories[i],na=False)])/total_non_na*100)
    print("For qualifier "+"2"+language_categories[i] +" - "+ formatted_float+"%")
    

Total Non-NA sub topics 41125
For qualifier 2A - 1.40%
For qualifier 2B - 0.00%
For qualifier 2C - 0.00%
For qualifier 2F - 0.00%
For qualifier 2G - 0.01%
For qualifier 2H - 0.00%
For qualifier 2J - 0.00%
For qualifier 2P - 0.00%
For qualifier 2S - 0.00%
For qualifier 2Z - 0.01%


In [10]:
#Further diving into the language qualifiers for sub topics
language_categories=["A","B","C","F","G","H","J","P","S","Z"]
total_non_na=len(items)-items.main_topic.isna().sum()
print("Total Non-NA main topics " + str(total_non_na))
for i in range(len(language_categories)):
    formatted_float = "{:.2f}".format(len(items[items["main_topic"].str.contains("2"+language_categories[i],na=False)])/total_non_na*100)
    print("For qualifier "+"2"+language_categories[i] +" - "+ formatted_float+"%")

Total Non-NA main topics 77772
For qualifier 2A - 0.00%
For qualifier 2B - 0.00%
For qualifier 2C - 0.00%
For qualifier 2F - 0.00%
For qualifier 2G - 0.00%
For qualifier 2H - 0.00%
For qualifier 2J - 0.00%
For qualifier 2P - 0.00%
For qualifier 2S - 0.00%
For qualifier 2Z - 0.00%


In [22]:
#for index, row in items.iterrows():
#    print(row.values[1])
items.query('itemID==47561')    

,itemID,title,author,publisher,main_topic,subtopics


In [23]:
#Creating the GET query
h = {'Authorization': '45942_59a4daad4956ef8fb1e0455f2800ec06'}
PARAMS={ 'page':1,'pageSize':1,'column':'title','beta':0}
url_substring="Testing It"
url_serialized=quote(url_substring)
final_url='https://api2.isbndb.com/books/' + url_serialized

try:
    response = req.get(final_url, timeout=5, headers=h,params=PARAMS)
    response.raise_for_status()
    # Code here will only run if the request is successful
except req.exceptions.HTTPError as errh:
    print(errh)
except req.exceptions.ConnectionError as errc:
    print(errc)
except req.exceptions.Timeout as errt:
    print(errt)
except req.exceptions.RequestException as err:
    print(err)

In [31]:
#Looping the above GET query for multiple queries
current_dat=pd.DataFrame(columns=['itemId','JsonString'])
error_id=pd.DataFrame(columns=['itemId','error_code'])
row_count=0
file_count=0

PARAMS={ 'page':1,'pageSize':5,'column':'title','beta':0}
for index, row in items.iterrows():
    url_substring=row.values[1]
    url_serialized=quote(url_substring)
    final_url='https://api2.isbndb.com/books/' + url_serialized
    
    try:
        response = req.get(final_url, headers=h,params=PARAMS)
        response.raise_for_status()
        
    except req.exceptions.HTTPError as errh:
        error_id=error_id.append({'itemId':row.values[0],'error_code':errh},ignore_index=True)
        print(errh)
        
    except req.exceptions.ConnectionError as errc:
        error_id=error_id.append({'itemId':row.values[0],'error_code':errc},ignore_index=True)
        print(errc)
        
    except req.exceptions.Timeout as errt:
        error_id=error_id.append({'itemId':row.values[0],'error_code':errt},ignore_index=True)
        print(errt)
        
    except req.exceptions.RequestException as err:
        error_id=error_id.append({'itemId':row.values[0],'error_code':err},ignore_index=True)
        print(err)
        
        
    current_dat=current_dat.append({'itemId':row.values[0], 'JsonString':response.json()},ignore_index=True)
    print("Written data for Item ID " + str(row.values[0]))
    row_count=row_count+1
    
    if(row_count==1000):
        current_dat.to_csv("data_current_run3_" + str(file_count) + ".csv")
        row_count=0
        file_count=file_count+1
        
    
        
        
    

Written data for Item ID 71208
Written data for Item ID 37181
Written data for Item ID 36246
Written data for Item ID 27706
Written data for Item ID 21726
Written data for Item ID 34275
Written data for Item ID 77135
Written data for Item ID 49606
Written data for Item ID 32692
Written data for Item ID 63393
Written data for Item ID 68030
Written data for Item ID 18448
Written data for Item ID 7632
Written data for Item ID 11725
Written data for Item ID 62763
Written data for Item ID 51329
Written data for Item ID 37027
Written data for Item ID 35243
Written data for Item ID 49779
Written data for Item ID 4755
Written data for Item ID 30584
Written data for Item ID 4485
Written data for Item ID 23711
Written data for Item ID 16353
Written data for Item ID 17649
Written data for Item ID 73285
Written data for Item ID 60177
Written data for Item ID 13760
Written data for Item ID 50161
Written data for Item ID 76283
Written data for Item ID 60368
Written data for Item ID 42671
Written dat

Written data for Item ID 72630
Written data for Item ID 77260
Written data for Item ID 70935
Written data for Item ID 53943
Written data for Item ID 8151
Written data for Item ID 69319
Written data for Item ID 25913
Written data for Item ID 69906
Written data for Item ID 8266
Written data for Item ID 33554
Written data for Item ID 11100
Written data for Item ID 41486
Written data for Item ID 15286
Written data for Item ID 43631
Written data for Item ID 5038
Written data for Item ID 32766
Written data for Item ID 42759
Written data for Item ID 77160
Written data for Item ID 31850
Written data for Item ID 13078
Written data for Item ID 17361
Written data for Item ID 15573
Written data for Item ID 73380
Written data for Item ID 2807
Written data for Item ID 29451
Written data for Item ID 10784
Written data for Item ID 42405
Written data for Item ID 12211
Written data for Item ID 75947
Written data for Item ID 5464
Written data for Item ID 12949
Written data for Item ID 58409
Written data 

Written data for Item ID 1281
Written data for Item ID 63995
Written data for Item ID 25530
Written data for Item ID 54683
Written data for Item ID 62671
Written data for Item ID 60160
404 Client Error: Not Found for url: https://api2.isbndb.com/books/S%C3%A9pharial?page=1&pageSize=5&column=title&beta=0
Written data for Item ID 31665
Written data for Item ID 61188
Written data for Item ID 45268
Written data for Item ID 68222
Written data for Item ID 2733
Written data for Item ID 71112
Written data for Item ID 50789
Written data for Item ID 59408
Written data for Item ID 76522
Written data for Item ID 18697
Written data for Item ID 22351
Written data for Item ID 72943
Written data for Item ID 41561
Written data for Item ID 58567
Written data for Item ID 26621
Written data for Item ID 58756
Written data for Item ID 27315
Written data for Item ID 6146
Written data for Item ID 7952
Written data for Item ID 56021
Written data for Item ID 53334
Written data for Item ID 72050
Written data for

Written data for Item ID 71205
Written data for Item ID 34051
Written data for Item ID 57269
Written data for Item ID 21076
Written data for Item ID 44239
Written data for Item ID 61576
Written data for Item ID 43991
Written data for Item ID 52800
Written data for Item ID 56395
Written data for Item ID 11501
Written data for Item ID 37247
Written data for Item ID 20005
Written data for Item ID 78230
Written data for Item ID 74273
Written data for Item ID 65481
Written data for Item ID 64512
Written data for Item ID 17184
Written data for Item ID 17433
Written data for Item ID 776
Written data for Item ID 23395
Written data for Item ID 31584
Written data for Item ID 2160
Written data for Item ID 8409
Written data for Item ID 67975
Written data for Item ID 32248
Written data for Item ID 8991
Written data for Item ID 48980
Written data for Item ID 41501
Written data for Item ID 17836
Written data for Item ID 3145
Written data for Item ID 77513
Written data for Item ID 37095
Written data f

Written data for Item ID 29790
Written data for Item ID 24956
Written data for Item ID 17655
Written data for Item ID 38272
Written data for Item ID 20165
Written data for Item ID 44190
Written data for Item ID 77577
Written data for Item ID 9064
Written data for Item ID 30474
Written data for Item ID 9809
Written data for Item ID 60555
Written data for Item ID 29326
Written data for Item ID 14113
Written data for Item ID 27479
Written data for Item ID 40249
Written data for Item ID 10349
Written data for Item ID 286
Written data for Item ID 42097
Written data for Item ID 43712
Written data for Item ID 38571
Written data for Item ID 58319
Written data for Item ID 4084
Written data for Item ID 5976
Written data for Item ID 45389
Written data for Item ID 44841
Written data for Item ID 11153
Written data for Item ID 72194
Written data for Item ID 22643
Written data for Item ID 17859
Written data for Item ID 65611
Written data for Item ID 30898
Written data for Item ID 47080
Written data f

...

Written data for Item ID 54705
403 Client Error: Forbidden for url: https://api2.isbndb.com/books/Philip%20K.%20Dick%3A%20Valis%20and%20Later%20Novels%20%28Loa%20%23193%29%3A%20A%20Maze%20of%20Death%20/%20Valis%20/%20The%20Divine%20Invasion%20/%20T?page=1&pageSize=5&column=title&beta=0
Written data for Item ID 62858
Written data for Item ID 3591
Written data for Item ID 78953
Written data for Item ID 52496
Written data for Item ID 8637
Written data for Item ID 57890
Written data for Item ID 2640
Written data for Item ID 17043
Written data for Item ID 32825
Written data for Item ID 13197
Written data for Item ID 10824
Written data for Item ID 36200
Written data for Item ID 63531
Written data for Item ID 76972
Written data for Item ID 71525
Written data for Item ID 5268
Written data for Item ID 2997
Written data for Item ID 8134
Written data for Item ID 75860
Written data for Item ID 34870
Written data for Item ID 50015
Written data for Item ID 19895
Written data for Item ID 55173
Writte

Written data for Item ID 43197
Written data for Item ID 61085
Written data for Item ID 47039
Written data for Item ID 1481
Written data for Item ID 42592
Written data for Item ID 29138
Written data for Item ID 53616
Written data for Item ID 66702
Written data for Item ID 7988
Written data for Item ID 18227
Written data for Item ID 53749
Written data for Item ID 31327
Written data for Item ID 28672
Written data for Item ID 32557
Written data for Item ID 11793
Written data for Item ID 29584
Written data for Item ID 10406
Written data for Item ID 23810
Written data for Item ID 17814
Written data for Item ID 12187
Written data for Item ID 19200
Written data for Item ID 36371
Written data for Item ID 1389
Written data for Item ID 46171
Written data for Item ID 45090
Written data for Item ID 63834
Written data for Item ID 1692
Written data for Item ID 21407
Written data for Item ID 33145
Written data for Item ID 28745
Written data for Item ID 61507
Written data for Item ID 76664
Written data

Written data for Item ID 66277
Written data for Item ID 77373
Written data for Item ID 72025
Written data for Item ID 57649
Written data for Item ID 24172
Written data for Item ID 7871
Written data for Item ID 42733
Written data for Item ID 9015
Written data for Item ID 6962
Written data for Item ID 26268
Written data for Item ID 53128
Written data for Item ID 12720
Written data for Item ID 43149
Written data for Item ID 53089
Written data for Item ID 58976
Written data for Item ID 4322
Written data for Item ID 44440
Written data for Item ID 68683
Written data for Item ID 49875
Written data for Item ID 66027
Written data for Item ID 60290
Written data for Item ID 53731
Written data for Item ID 34771
Written data for Item ID 12525
Written data for Item ID 1282
Written data for Item ID 54118
Written data for Item ID 43014
Written data for Item ID 21249
Written data for Item ID 13924
Written data for Item ID 42725
Written data for Item ID 29887
Written data for Item ID 29015
Written data 

Written data for Item ID 10017
Written data for Item ID 54923
Written data for Item ID 64746
Written data for Item ID 44773
Written data for Item ID 40027
Written data for Item ID 46703
Written data for Item ID 60240
Written data for Item ID 70039
Written data for Item ID 36177
Written data for Item ID 55289
Written data for Item ID 28347
Written data for Item ID 48818
Written data for Item ID 6919
Written data for Item ID 26390
Written data for Item ID 3298
Written data for Item ID 37821
Written data for Item ID 19486
Written data for Item ID 76562
Written data for Item ID 28677
Written data for Item ID 57391
Written data for Item ID 7315
Written data for Item ID 16018
Written data for Item ID 12512
Written data for Item ID 27771
Written data for Item ID 6209
Written data for Item ID 11881
Written data for Item ID 24986
Written data for Item ID 49664
Written data for Item ID 43046
Written data for Item ID 51260
Written data for Item ID 62771
Written data for Item ID 76081
Written data

Written data for Item ID 1801
Written data for Item ID 73494
Written data for Item ID 75998
Written data for Item ID 63904
Written data for Item ID 37942
Written data for Item ID 58386
Written data for Item ID 26442
Written data for Item ID 38942
Written data for Item ID 44665
Written data for Item ID 36868
Written data for Item ID 78053
Written data for Item ID 74904
Written data for Item ID 30800
Written data for Item ID 67662
Written data for Item ID 70592
Written data for Item ID 23445
Written data for Item ID 7553
Written data for Item ID 50174
Written data for Item ID 32715
Written data for Item ID 76840
Written data for Item ID 65168
Written data for Item ID 30336
Written data for Item ID 2916
Written data for Item ID 32577
Written data for Item ID 6965
Written data for Item ID 5602
Written data for Item ID 31449
Written data for Item ID 57779
Written data for Item ID 44780
Written data for Item ID 34419
Written data for Item ID 30305
Written data for Item ID 48135
Written data 

In [27]:
current_dat.head()

,itemId,JsonString
0,3509,"{'total': 10000, 'books': [{'title': 'Eliel Sa..."
1,4613,"{'total': 10000, 'books': [{'title': 'Die Lüge..."
2,2078,"{'total': 10000, 'books': [{'title': 'Samtpfot..."
3,46735,"{'total': 10000, 'books': [{'title': 'Whistlin..."
4,77081,"{'total': 10000, 'books': [{'title': 'Invisibl..."


In [28]:
current_dat.to_csv("data_current_run3_manual.csv")

In [27]:
api_dat=pd.read_excel("final_api_data.xlsx")

In [28]:
api_dat.drop(api_dat[api_dat['JsonString'].str.contains("errorType",case=False)].index,inplace=True)

In [29]:
api_dat.to_excel("api_data_modified.xlsx")

In [24]:
api_dat['title']=api_dat['JsonString'].str[api_dat['JsonString'].str.find('title')

In [38]:
for index, row in api_dat.iterrows():
    #print(row.values[1])
    json_dat=str(json.loads(row.values[1]))
#api_dat['loading']=json.loads(api_dat['JsonString'].str)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)